##ThinkDSP

This notebook contains code solutions to exercises in Chapter 5: Autocorrelation

Copyright 2015 Allen Downey

License: [Creative Commons Attribution 4.0 International](http://creativecommons.org/licenses/by/4.0/)

In [ ]:
from __future__ import print_function, division

import thinkdsp
import thinkplot
import thinkstats2

import numpy as np
import pandas as pd

%matplotlib inline

**Exercise:** If you did the exercises in the previous chapter, you downloaded
the historical price of BitCoins and estimated the power spectrum
of the price changes.  Using the same data, compute the autocorrelation
of BitCoin prices.  Does the autocorrelation function drop off quickly?  Is there evidence of periodic behavior?

In [ ]:
df = pd.read_csv('coindesk-bpi-USD-close.csv', nrows=1625, parse_dates=[0])
ys = df.Close.values

In [ ]:
wave = thinkdsp.Wave(ys, framerate=1)
wave.plot()

Here's the autocorrelation function using the statistical definition, which unbiases, normalizes, and standardizes; that is, it shifts the mean to zero, divides through by standard deviation, and divides the sum by N.

In [ ]:
from autocorr import autocorr

lags, corrs = autocorr(wave)

thinkplot.preplot(5)
thinkplot.plot(lags, corrs)
thinkplot.config(xlabel='lag',
                 ylabel='correlation')

The ACF drops off slowly with increasing lag, suggesting some kind of pink noise.  And it looks like there are moderate correlations with lags near 200, 425 and 700 days.

We can compare my implementation of `autocorr` with `np.correlate`, which uses the definition of correlation used in signal processing.  It doesn't unbias, normalize, or standardize the wave.

In [ ]:
corrs2 = np.correlate(wave.ys, wave.ys, mode='same')

thinkplot.preplot(1)
thinkplot.plot(corrs2)
thinkplot.config(xlabel='lag',
                 ylabel='dot product')

The second half of the result corresponds to positive lags:

In [ ]:
N = len(corrs2)
half = corrs2[N//2:]
thinkplot.plot(half)
thinkplot.config(xlabel='lag',
                 ylabel='dot product')

We can standardize the results after the fact by dividing through by `lengths`:

In [ ]:
lengths = range(N, N//2, -1)
half /= lengths
half /= half[0]
thinkplot.plot(half)
thinkplot.config(xlabel='lag',
                 ylabel='dot product')

But even after standardizing, the results look very different.  In the results from `correlate`, the peak at lag 200 is less apparent, and the other two peaks are obliterated.

In [ ]:
thinkplot.preplot(2)
thinkplot.plot(corrs, label='autocorr')
thinkplot.plot(half, label='correlate')
thinkplot.config(xlabel='lag', ylabel='correlation')

I think the reason the results are so different is that this dataset looks very different in different parts of the range; in particular, the variance changes a lot over time.

For this dataset, the statistical definition of ACF, is probably more appropriate.


**Exercise:** The example code in `chap05.ipynb` shows how to use autocorrelation
to estimate the fundamental frequency of a periodic signal.
Encapsulate this code in a function called `estimate_fundamental`,
and use it to track the pitch of a recorded sound.

To see how well it works, try superimposing your pitch estimates on a
spectrogram of the recording.

In [ ]:
wave = thinkdsp.read_wave('28042__bcjordan__voicedownbew.wav')
wave.normalize()
wave.make_audio()

I'll use the same example from `chap05.ipynb`.  Here's the spectrogram:

In [ ]:
wave.make_spectrogram(4096).plot(high=70)

And here's a function that encapsulates the code from Chapter 5.  In general, finding the first, highest peak in the autocorrelation function is tricky.  I kept is simple by specifying the range of lags to search.

In [ ]:
def estimate_fundamental(segment, low=70, high=150):
    lags, corrs = autocorr(segment)
    lag = np.array(corrs[low:high]).argmax() + low
    period = lag / segment.framerate
    frequency = 1 / period
    return frequency

Here's an example of how it works.

In [ ]:
duration = 0.01
segment = wave.segment(start=0.2, duration=duration)
freq = estimate_fundamental(segment)
freq

And here's a loop that tracks pitch over the sample.

The `ts` are the mid-points of each segment.

In [ ]:
step = 0.05
starts = np.arange(0.0, 1.4, step)

ts = []
freqs = []

for start in starts:
    ts.append(start + step/2)
    segment = wave.segment(start=start, duration=duration)
    freq = estimate_fundamental(segment)
    freqs.append(freq)

Here's the pitch-tracking curve superimposed on the spectrogram:

In [ ]:
wave.make_spectrogram(4096).plot(high=70)
thinkplot.plot(ts, freqs, color='green')

Looks pretty good!